In [ ]:
import os 
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from ruamel.yaml import YAML
from collections import defaultdict

## Images preparation

In [2]:
# df = pd.read_csv('/raid/immucan/IMC/immu-vis-other-ds/full_images_list-8.06.2025.csv', index_col=0)
df = pd.read_csv('/raid/immucan/IMC/immu-vis-other-ds/full_images_only_markers_list-17.06.2025.csv', index_col=0)
df

,tiff_path,new_tiff_path,shape,dataset,dataset_subtype
0,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 744, 889)",bowen-lynch,bowen-lynch
1,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 800, 800)",bowen-lynch,bowen-lynch
2,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 525, 1141)",bowen-lynch,bowen-lynch
3,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 378, 986)",bowen-lynch,bowen-lynch
4,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 687, 940)",bowen-lynch,bowen-lynch
...,...,...,...,...,...
6109,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 947, 1038)",hoch-rna,hoch-rna
6110,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 936, 1010)",hoch-rna,hoch-rna
6111,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 884, 909)",hoch-rna,hoch-rna
6112,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 818, 1117)",hoch-rna,hoch-rna


In [3]:
# remove too small images
df['shape'] = df['shape'].apply(lambda x: eval(x))
df = df[df['shape'].apply(lambda x: x[1] > 112 and x[2] > 112)]

df

,tiff_path,new_tiff_path,shape,dataset,dataset_subtype
0,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 744, 889)",bowen-lynch,bowen-lynch
1,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 800, 800)",bowen-lynch,bowen-lynch
2,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 525, 1141)",bowen-lynch,bowen-lynch
3,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 378, 986)",bowen-lynch,bowen-lynch
4,/raid/immucan/IMC/immu-vis-other-ds/bowen-lync...,/raid/immucan/IMC/immu-vis-other-ds/full/bowen...,"(39, 687, 940)",bowen-lynch,bowen-lynch
...,...,...,...,...,...
6109,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 947, 1038)",hoch-rna,hoch-rna
6110,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 936, 1010)",hoch-rna,hoch-rna
6111,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 884, 909)",hoch-rna,hoch-rna
6112,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 818, 1117)",hoch-rna,hoch-rna


In [4]:
df.groupby(['dataset', 'dataset_subtype'])['new_tiff_path'].count()

dataset       dataset_subtype
ajaib         ajaib               34
bengsch       bengsch            108
bowen-lynch   bowen-lynch         85
damond        damond-lung        845
danenberg     danenberg          793
haley_glio    haley_glio          24
hoch-protein  hoch-protein       167
hoch-rna      hoch-rna           166
immucan-p1    bc2                813
              hn                 743
              nsclc2             744
immucan-p2    nsclc2             566
jackson       jackson-basel      380
              jackson-zurich     365
kucukkose     kucukkose           27
ohara         ohara               58
susman        susman              35
xu            xu                 158
Name: new_tiff_path, dtype: int64

In [5]:
# make distinct dataset_subtype for two nsclc2 panels
df.loc[df['dataset'] == 'immucan-p2', 'dataset_subtype'] = 'nsclc2-panel2'
df.loc[(df['dataset'] == 'immucan-p1') & (df['dataset_subtype'] == 'nsclc2'), 'dataset_subtype'] = 'nsclc2-panel1'

In [6]:
df['dataset_subtype'].value_counts()

dataset_subtype
damond-lung       845
bc2               813
danenberg         793
nsclc2-panel1     744
hn                743
nsclc2-panel2     566
jackson-basel     380
jackson-zurich    365
hoch-protein      167
hoch-rna          166
xu                158
bengsch           108
bowen-lynch        85
ohara              58
susman             35
ajaib              34
kucukkose          27
haley_glio         24
Name: count, dtype: int64

In [7]:
# train test split

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(splitter.split(df, df['dataset_subtype']))

train_paths = df.iloc[train_idx]['new_tiff_path'].values
test_paths = df.iloc[test_idx]['new_tiff_path'].values

train_split = df.iloc[train_idx]
test_split = df.iloc[test_idx]

In [8]:
train_split

,tiff_path,new_tiff_path,shape,dataset,dataset_subtype
4119,/raid/immucan/IMC/NSCLC2/IMC1/unzipped/train/L...,/raid/immucan/IMC/immu-vis-other-ds/full/immuc...,"(40, 600, 600)",immucan-p1,nsclc2-panel1
2168,/raid/immucan/IMC/immu-vis-other-ds/danenberg_...,/raid/immucan/IMC/immu-vis-other-ds/full/danen...,"(39, 511, 591)",danenberg,danenberg
5880,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(46, 1043, 904)",hoch-protein,hoch-protein
4320,/raid/immucan/IMC/NSCLC2/IMC1/unzipped/test/LU...,/raid/immucan/IMC/immu-vis-other-ds/full/immuc...,"(40, 600, 600)",immucan-p1,nsclc2-panel1
1764,/raid/immucan/IMC/immu-vis-other-ds/danenberg_...,/raid/immucan/IMC/immu-vis-other-ds/full/danen...,"(39, 628, 534)",danenberg,danenberg
...,...,...,...,...,...
6019,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 941, 1122)",hoch-rna,hoch-rna
6067,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 941, 1043)",hoch-rna,hoch-rna
1161,/raid/immucan/IMC/immu-vis-other-ds/damond_dia...,/raid/immucan/IMC/immu-vis-other-ds/full/damon...,"(37, 318, 338)",damond,damond-lung
2839,/raid/immucan/IMC/immu-vis-other-ds/bengsch/DC...,/raid/immucan/IMC/immu-vis-other-ds/full/bengs...,"(42, 1300, 1300)",bengsch,bengsch


In [9]:
test_split

,tiff_path,new_tiff_path,shape,dataset,dataset_subtype
1442,/raid/immucan/IMC/immu-vis-other-ds/damond_dia...,/raid/immucan/IMC/immu-vis-other-ds/full/damon...,"(37, 340, 394)",damond,damond-lung
4982,/raid/immucan/IMC/BC2NEW/BC2/IMC1/zip/img/IMMU...,/raid/immucan/IMC/immu-vis-other-ds/full/immuc...,"(40, 600, 600)",immucan-p1,bc2
3354,/raid/immucan/IMC/SHCN/IMC1/zip/img/IMMUcan_Ba...,/raid/immucan/IMC/immu-vis-other-ds/full/immuc...,"(40, 600, 600)",immucan-p1,hn
5810,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(46, 1012, 898)",hoch-protein,hoch-protein
2067,/raid/immucan/IMC/immu-vis-other-ds/danenberg_...,/raid/immucan/IMC/immu-vis-other-ds/full/danen...,"(39, 722, 568)",danenberg,danenberg
...,...,...,...,...,...
5988,/raid/immucan/IMC/immu-vis-other-ds/hoch/full_...,/raid/immucan/IMC/immu-vis-other-ds/full/hoch-...,"(41, 992, 1140)",hoch-rna,hoch-rna
267,/raid/immucan/IMC/immu-vis-other-ds/jackson_br...,/raid/immucan/IMC/immu-vis-other-ds/full/jacks...,"(52, 688, 762)",jackson,jackson-basel
3111,/raid/immucan/IMC/SHCN/IMC1/zip/img/IMMUcan_Ba...,/raid/immucan/IMC/immu-vis-other-ds/full/immuc...,"(40, 600, 600)",immucan-p1,hn
2183,/raid/immucan/IMC/immu-vis-other-ds/danenberg_...,/raid/immucan/IMC/immu-vis-other-ds/full/danen...,"(39, 617, 501)",danenberg,danenberg


In [11]:
# make symlinks to the images in the respective split directories
split_map = {
    'train': train_split,
    'test': test_split
}

output_dir = './data' # your output directory path
for split_name, split_df in split_map.items():
    split_dir = os.path.join(output_dir, split_name)
    os.makedirs(split_dir, exist_ok=True)
    
    # Save the split DataFrame to a CSV file
    split_df.to_csv(os.path.join(split_dir, 'images_list.csv'), index=False)
    
    # Copy the image files to the respective split directory
    for img_path, subdir in split_df[['new_tiff_path', 'dataset_subtype']].values:
        # Create subdirectory for dataset subtype if it doesn't exist
        subdir_path = os.path.join(split_dir, subdir, 'imgs')
        os.makedirs(subdir_path, exist_ok=True)

        # create symlink to the image file
        output_filename = img_path.split('/', 5)[-1].replace('/', '.')
        trg_path = os.path.join(subdir_path, output_filename)
        os.symlink(img_path, trg_path)

## Marker mapping preparation

### If you are using already existing markers configs (default all_markers_tokenizer.yaml and all_panels_config.yaml) you can skip the below code.

In [10]:
marker_df = pd.read_csv('/raid/immucan/IMC/immu-vis-other-ds/marker_assignment_08.06.2025.csv', index_col=0)
marker_df

,Marker Name,Comment,Aliases,hoch-protein,hoch-rna,xu,kucukkose,bengsch,jackson,ohara,bowen-lynch,susman,ajaib,immucan-p1,immucan-p2,damond,danenberg,haley_glio
0,AKT,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMY2A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN
2,Annexin A2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN
3,Arg1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
4,AXL,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Twist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,VEGF,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,Vimentin,NaN,NaN,16.0,1.0,5.0,NaN,NaN,20.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
201,VISTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,30.0,NaN,NaN,NaN,21.0


In [14]:
marker_tokenizer = {
    marker: i for i, marker in enumerate(marker_df['Marker Name'].values)
}
out_yaml = './configs/all_markers_tokenizer.yaml' # your path to all_markers_tokenizer.yaml
with open(out_yaml, 'w') as f:
    YAML().dump(marker_tokenizer, f)

In [15]:
dataset_names = marker_df.columns[3:].values
panel_tokenizer = defaultdict(dict)
for marker, datasets in zip(marker_df['Marker Name'], marker_df.iloc[:, 3:].values):
    # print(marker, datasets)
    for i, d in enumerate(datasets):
        if pd.isna(d):
            continue
        # print(f'{marker} -> {dataset_names[i]}: {d}')
        panel_tokenizer[dataset_names[i]][int(d)] = marker


In [ ]:
# for each dataset, sort the dict by keys and convert values to a list
panel_yaml = {'markers' : {}}
for dataset, markers in panel_tokenizer.items():
    sorted_markers = dict(sorted(markers.items()))
    print(f'{dataset}: {sorted_markers}')

    panel_yaml['markers'][dataset] = list(sorted_markers.values())

out_yaml = './configs/all_panels_config.yaml'
with open(out_yaml, 'w') as f:
    YAML().dump(panel_yaml, f)

kucukkose: {5: 'SMA', 6: 'Ecad', 7: 'CD45', 8: 'CD14', 9: 'TBET', 10: 'CD16', 11: 'beta-Catenin', 12: 'panCK', 13: 'CTLA-4', 14: 'PDL1', 15: 'IFNy', 16: 'CD45RO', 17: 'AKT', 18: 'LAG3', 19: 'FOXP3', 20: 'CD4', 21: 'pSTAT3', 22: 'CD68', 23: 'IL10', 24: 'CD20', 25: 'CD8a', 26: 'ICOS', 27: 'pS6', 28: 'PD1', 29: 'NFkb', 30: 'IL17a', 31: 'Ki67', 32: 'GrzB', 33: 'CD3', 34: 'pERK', 35: 'c-Caspase3c-PARP', 36: 'ERK', 37: 'pAKT', 38: 'Histone H3'}
damond: {0: 'Histone H3', 1: 'SMA', 2: 'Insulin', 3: 'CD38', 4: 'CD44', 5: 'PCSK2', 6: 'CD99', 7: 'CD68', 8: 'MPO', 9: 'GLUT1', 10: 'CD20', 11: 'AMY2A', 12: 'CD3', 13: 'PPY', 14: 'PIN', 15: 'GCG', 16: 'PDX1', 17: 'SST', 18: 'SYP', 19: 'CK19', 20: 'CD45', 21: 'FOXP3', 22: 'CD45RA', 23: 'CD8a', 24: 'Carbonic Anhydrase', 25: 'IAPP', 26: 'Ki67', 27: 'NKX6-1', 28: 'Histone H3 - p', 29: 'CD4', 30: 'CD31', 31: 'Ecad', 32: 'PTPRN', 33: 'pRB', 34: 'c-Caspase3c-PARP', 35: 'DNA1', 36: 'DNA2'}
ajaib: {0: 'JARID2-C', 1: 'NKp46', 2: 'EZH2', 3: 'SOD2', 4: 'NeuN', 5: